In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
# сделаем нормирование значений пикселей (поделим на 255)
X_train = X_train / 255
X_test = X_train / 255

In [3]:
cnn = models.Sequential([
    
    # cnn layers 
    # filters - число фичей, которые мы будем выделять в изображении.
    # kernel size - размер фичи
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)),
    
    # pooling - заменяем квадрат пикселей 2х2 на один пиксель argmax по квадрату 
    layers.MaxPooling2D((2,2)),
    
    
    # dense layers 
    layers.Flatten(input_shape=(32,32,3)),
    # слой ниже будем заменять TT слоем
    layers.Dense(1024, activation='relu'),
    layers.Dense(3125, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [4]:
cnn.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [5]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 1024)              7373824   
                                                                 
 dense_1 (Dense)             (None, 3125)              3203125   
                                                                 
 dense_2 (Dense)             (None, 10)                31260     
                                                        

In [6]:
cnn.fit(X_train, y_train, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 159s 101ms/step - loss: 1.4083 - accuracy: 0.4909
Epoch 2/3
1563/1563 [==============================] - 157s 101ms/step - loss: 1.0561 - accuracy: 0.6263
Epoch 3/3
1563/1563 [==============================] - 159s 102ms/step - loss: 0.8755 - accuracy: 0.6925


In [7]:
import t3f
# W1 = cnn.trainable_weights[2]
# print(W1.shape)
# Wtt1 = t3f.to_tt_matrix(W1, shape=[[4, 4, 4, 4, 4], [5, 5, 5, 5, 5]], max_tt_rank=8)
# print(Wtt1)

W2 = cnn.trainable_weights[4]
print(W2.shape)
Wtt2 = t3f.to_tt_matrix(W2, shape=[[4, 4, 4, 4, 4], [5, 5, 5, 5, 5]], max_tt_rank=8)
print(Wtt2)

(1024, 3125)
A TT-Matrix of size 1024 x 3125, underlying tensor shape: (4, 4, 4, 4, 4) x (5, 5, 5, 5, 5), TT-ranks: (1, 8, 8, 8, 8, 1)


In [8]:
cores = Wtt.tt_cores
other_params = cnn.get_weights()[4:]

NameError: name 'Wtt' is not defined

## Заменим на TT-слой

In [10]:
cnn_tt = models.Sequential([
    
    # cnn layers 
    # filters - число фичей, которые мы будем выделять в изображении.
    # kernel size - размер фичи
    layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(32,32,3)),
    
    # pooling - заменяем квадрат пикселей 2х2 на один пиксель argmax по квадрату 
    layers.MaxPooling2D((2,2)),
    
    
    # dense layers 
    layers.Flatten(input_shape=(32,32,3)),
    layers.Dense(1024, activation='relu'),
    t3f.nn.KerasDense(input_dims=[4, 4, 4, 4, 4], output_dims=[5, 5, 5, 5, 5], tt_rank=8, activation='relu'),
    layers.Dense(10, activation='softmax')
])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
cnn_tt.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [12]:
cnn_tt.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 7200)              0         
                                                                 
 dense_3 (Dense)             (None, 1024)              7373824   
                                                                 
 tt_dense_0 (KerasDense)     (None, 3125)              7285      
                                                                 
 dense_4 (Dense)             (None, 10)                31260     
                                                      

In [13]:
cnn_tt.fit(X_train, y_train, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 157s 100ms/step - loss: 1.3693 - accuracy: 0.5070
Epoch 2/3
1563/1563 [==============================] - 158s 101ms/step - loss: 0.9875 - accuracy: 0.6514
Epoch 3/3
1563/1563 [==============================] - 156s 100ms/step - loss: 0.7713 - accuracy: 0.7271
